In [4]:
import numpy as np
import pandas as pd
from datetime import datetime
import json

import data as Data
from indicators import *
import signals as Signal
from portfolio import Portfolio

data = Data.get_candles('db', [1], 1440)
data

,open,high,low,close,volume
,EURUSD,EURUSD,EURUSD,EURUSD,EURUSD
timestamp,,,,,
2023-12-26,1.10230,1.10451,1.10086,1.10422,0.0
2023-12-27,1.10410,1.11229,1.10285,1.11062,0.0
2023-12-28,1.11028,1.11395,1.10551,1.10619,0.0
2023-12-29,1.10610,1.10844,1.10340,1.10343,0.0
2024-01-02,1.10429,1.10442,1.09382,1.09410,0.0
...,...,...,...,...,...
2024-05-06,1.07601,1.07908,1.07553,1.07700,0.0
2024-05-07,1.07651,1.07874,1.07478,1.07500,38.0


In [6]:
sma = SMA(data, window=10, source='close').run()
sma_reset = sma.reset_index()

# Convert the timestamp column to string (ISO 8601 format)
sma_reset['timestamp'] = sma_reset['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Now, convert the DataFrame to a list of dictionaries (JSON serializable)
x = sma_reset.to_dict(orient="records")

# Serialize the data to JSON (with NaN values as null)
j = json.dumps(x, allow_nan=True)
j

'[{"timestamp": "2023-12-26 00:00:00", "EURUSD": NaN}, {"timestamp": "2023-12-27 00:00:00", "EURUSD": NaN}, {"timestamp": "2023-12-28 00:00:00", "EURUSD": NaN}, {"timestamp": "2023-12-29 00:00:00", "EURUSD": NaN}, {"timestamp": "2024-01-02 00:00:00", "EURUSD": NaN}, {"timestamp": "2024-01-03 00:00:00", "EURUSD": NaN}, {"timestamp": "2024-01-04 00:00:00", "EURUSD": NaN}, {"timestamp": "2024-01-05 00:00:00", "EURUSD": NaN}, {"timestamp": "2024-01-08 00:00:00", "EURUSD": NaN}, {"timestamp": "2024-01-09 00:00:00", "EURUSD": 1.0987449999999999}, {"timestamp": "2024-01-10 00:00:00", "EURUSD": 1.098043}, {"timestamp": "2024-01-11 00:00:00", "EURUSD": 1.096712}, {"timestamp": "2024-01-12 00:00:00", "EURUSD": 1.095589}, {"timestamp": "2024-01-15 00:00:00", "EURUSD": 1.094746}, {"timestamp": "2024-01-16 00:00:00", "EURUSD": 1.0940869999999998}, {"timestamp": "2024-01-17 00:00:00", "EURUSD": 1.093684}, {"timestamp": "2024-01-18 00:00:00", "EURUSD": 1.092981}, {"timestamp": "2024-01-19 00:00:00", 

In [ ]:
close_above_sma = Signal.above(data['close'], sma['SMA_20'])
crossover = Signal.crossover(sma['SMA_10'], sma['SMA_20'])

buy_signals = crossover & close_above_sma
buy_signals

In [ ]:
crossunder = Signal.crossunder(sma['SMA_10'], sma['SMA_20'])

sell_signals = crossunder
sell_signals

In [ ]:
pf = Portfolio.from_signals(data, buy_signals, sell_signals)
pf.get_stats()

# MAIN TEST

In [ ]:
import asyncio
import websockets
import json
import threading

In [ ]:
async def on_message(message):
    print(f"Received: {message}")

async def websocket_client():
    uri = "ws://172.21.0.5:8765"
    print('connect')
    # Connect to the server
    async with websockets.connect(uri) as websocket:
        print("connected")
        # Send a message
        message = {
            'type': 'Login',
            'name': 'Backtester'
        }
        message = json.dumps(message)
        await websocket.send(message)
        print(f"Sent: {message}")

        # Receive messages
        async for message in websocket:
            await on_message(message)

def run_server_in_thread():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(websocket_client())

In [ ]:
client_thread = threading.Thread(target=run_server_in_thread)
client_thread.start()